In [2]:
from config import get_model_client_4o, get_model_client_41
from IPython.display import Markdown, display

In [3]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat, SelectorGroupChat, MagenticOneGroupChat
from autogen_agentchat.ui import Console
from typing import Callable


def create_and_get_assistant_agent(
        name: str,
        description: str,
        system_message: str,
        tools: list[Callable] = [],
        model_client: str = '4o'
    ) -> AssistantAgent:
    if model_client == '4o':
        model_client = get_model_client_4o()
    elif model_client == '41':
        model_client = get_model_client_41()    
    return AssistantAgent(
        name=name,
        description=description,
        model_client=model_client,
        system_message=system_message,
        tools=tools
    )


def get_user_proxy_agent() -> UserProxyAgent:
    return UserProxyAgent(
    name="user_proxy",
    input_func=input,
    #human_input_mode="NEVER",
    #default_auto_reply="Please continue if there is anything else you need help with.",
    #max_turns=10
    )

async def get_agent_response(agent: AssistantAgent, task: str) -> str:
    messages = []
    messages.append(TextMessage(content=task, source="user"))
    response = await agent.on_messages(
    messages, CancellationToken()
    )
    return response.chat_message.content


## Create an Agent and ask a question

In [10]:
geography_agent = create_and_get_assistant_agent(
    name="geography_agent",
    description="A helpful assistant that can answer questions about geography.",
    system_message="You are a helpful assistant that can answer questions about geography."
)

response = await get_agent_response(geography_agent, "What is the capital of France?")
display(Markdown(response))

/Users/andreakrogdal/repos/moe-workshop/.venv/lib/python3.12/site-packages/autogen_ext/models/openai/_openai_client.py:413: UserWarning: Missing required field 'structured_output' in ModelInfo. This field will be required in a future version of AutoGen.
  validate_model_info(self._model_info)


The capital of France is **Paris**.

## Create a team with an agent and a human in the loop

### Adding function tools which the agent uses to get correct information


In [4]:
async def get_weather(location: str) -> str:
    """
   Return the current weather based on location
    """
    if location == "Oslo":          
        return "The weather in Oslo is sunny."
    elif location == "Stockholm":
        return "The weather in Stockholm is cloudy."
    else:
        return f"Sorry, I don't know the weather in {location}."

async def get_news(location: str) -> str:
    """
    Return the latest news based on location
    """
    if location == "Oslo":
        return "Nothing special is happening in Oslo."
    elif location == "Stockholm":
        return "It is a lot of happening in Stockholm. The king is going to visit the city and Lady Gaga is having a concert in Avicii Arena"
    else:
        return f"Sorry, I don't know the news in {location}."


### Define Agent with the tools

In [ ]:
nordic_travel_agent = create_and_get_assistant_agent(
    name="nordic_travel_agent",
    description="A helpful assistant that can answer questions about travel in the Nordic countries.",
    system_message="You are a helpful assistant that can answer questions about travel in the Nordic countries.",
    tools=[get_weather, get_news]
)


/Users/andreakrogdal/repos/moe-workshop/.venv/lib/python3.12/site-packages/autogen_ext/models/openai/_openai_client.py:413: UserWarning: Missing required field 'structured_output' in ModelInfo. This field will be required in a future version of AutoGen.
  validate_model_info(self._model_info)


### Create a team with the agent and user.

In [ ]:
termination = TextMentionTermination("DONE")

nordic_team = SelectorGroupChat(
    participants=[
        nordic_travel_agent,
        get_user_proxy_agent()
    ],
    model_client=get_model_client_4o(),
    max_turns=15,
    termination_condition=termination
)

### Chat with the travel expert to decide where to travel :)

In [ ]:
task = "Im going for a trip to either Stockholm or Berlin. Can you give me some information about them which will make it easier for me to decide based on weather and news?"

response = await Console(nordic_team.run_stream(task=task))